In [ ]:
import tensorflow as tf
import numpy as np

# Computing CKA

In [ ]:
randgen = np.random.default_rng(seed=0)
randgen.choice(np.arange(100),10)


In [ ]:
sample_features_x = np.random.random([10000,16384])
sample_features_y = np.random.random([10000,16384])

In [ ]:
def feature_space_linear_cka(features_x, features_y):
    features_x = features_x - np.mean(features_x, 0, keepdims=True)
    features_y = features_y - np.mean(features_y, 0, keepdims=True)

    dot_product_similarity = np.linalg.norm(features_x.T.dot(features_y)) ** 2
    normalization_x = np.linalg.norm(features_x.T.dot(features_x))
    normalization_y = np.linalg.norm(features_y.T.dot(features_y))

    return dot_product_similarity / (normalization_x * normalization_y)

print(feature_space_linear_cka(sample_features_x, sample_features_y))


In [ ]:
tf.experimental.numpy.experimental_enable_numpy_behavior()

def feature_space_linear_cka_tf(features_x, features_y):
    features_x = features_x - tf.math.reduce_mean(features_x, axis=0,keepdims=True)
    features_y = features_y - tf.math.reduce_mean(features_y, axis=0,keepdims=True)

    product_crossed = tf.matmul(features_x.transpose(), features_y)
    dot_product_similarity = tf.linalg.norm(product_crossed) ** 2
    gram_x = tf.matmul(features_x.transpose(), features_x)
    normalization_x = tf.linalg.norm(gram_x)
    gram_y = tf.matmul(features_y.transpose(), features_y)
    normalization_y = tf.linalg.norm(gram_y)

    return dot_product_similarity / (normalization_x * normalization_y)


print(feature_space_linear_cka_tf(sample_features_x, sample_features_y))

# Test model

In [ ]:
path_model = 'pruning_history/model_1_00_unpruned.keras'
model = tf.keras.models.load_model(path_model).layers[-1]

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

X_train_mean = np.mean(X_train, axis=0)
X_train -= X_train_mean
X_test -= X_train_mean

In [ ]:
y_test_predicted_probs = model.predict(X_test)
y_test_predicted = np.argmax(y_test_predicted_probs, axis=1)

In [ ]:
metric = tf.keras.metrics.Accuracy()
metric(y_test_predicted, y_test[:,0]).numpy()

# Evaluate scores

In [ ]:
from main import CKA
import rebuild_layers as rl

allowed_layers = rl.blocks_to_prune(model)
layer_method = CKA(method='intra_crossed')
scores = layer_method.scores(model, X_test, y_test, allowed_layers)

In [ ]:
similarty_scores = np.copy(scores[1])

for i in range(len(similarty_scores)):
    for j in range(len(similarty_scores)):
        similarty_scores[j, i] = similarty_scores[i, j]

np.save(
    'similarty_scores.npy',
    similarty_scores
)

In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
from matplotlib import colormaps

A = similarty_scores / np.max(similarty_scores)

mask =  np.tri(A.shape[0], k=0)
A = np.ma.array(A, mask=mask) # mask out the lower triangle

fig = plt.figure()
ax = fig.add_subplot(111)
cmap = colormaps['viridis']
cmap.set_bad('w')
plot=ax.pcolor(A)
plt.tick_params(axis='x',which='both',bottom=False, top=False, labelbottom=False)
plt.tick_params(axis='y',which='both',left=False, right=False, labelleft=False)

fig.colorbar(plot)

plt.show()


# Comparative plot

In [ ]:
from pathlib import Path
dir_models = Path('./pruning_history_third_succes/')

paths_models = [f for f in dir_models.glob('*') if f.suffix == '.keras']
paths_models = np.sort(paths_models)
n_models = len(paths_models)

In [ ]:
from main import compute_flops

flops = np.zeros([n_models])
n_param = np.zeros([n_models])
accuracies = np.zeros([n_models])

get_accuracy = tf.keras.metrics.Accuracy()

y_test_expected = y_test[:, 0]

for i, path_model in enumerate(paths_models):
    
    model = tf.keras.models.load_model(str(path_model))
    model = model.layers[-1]

    flops[i] = compute_flops(model)[0]
    
    y_test_predicted_probs = model.predict(X_test)
    y_test_predicted = np.argmax(y_test_predicted_probs, axis=1)
    
    accuracies[i] = get_accuracy(y_test_expected, y_test_predicted).numpy()


In [ ]:
np.save(
    'models_performance.npy',
    {
        'flops': flops,
        'accuracies': accuracies
    }
)

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

percentual_flop_drop = (1 - flops / flops[0])
delta_in_accuracy = accuracies / accuracies[0] - 1

y_min, y_max = -2.0, 2.0
y_ticks = np.linspace(y_min, y_max, 5)

plt.plot(percentual_flop_drop * 100, delta_in_accuracy * 100, '-x')

plt.ylim([y_min, y_max])
plt.xlim([0.0, 100.0])

plt.yticks(y_ticks)

plt.grid(linestyle='dashed')

plt.ylabel('$\Delta$ in accuracy [pp]')
plt.xlabel('FLOPS drop [%]')

# Plot training

In [ ]:
import pandas as pd

training_data = pd.read_csv('pruning_history_third_succes/model_1_00_unpruned.log')

In [ ]:
%matplotlib qt

plt.plot(training_data['accuracy'])
plt.plot(training_data['val_accuracy'])

plt.legend(['Train', 'Validation'], loc='center right')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
